<a href="https://colab.research.google.com/github/femolinas/Projeto_Acha_Preco/blob/main/Projeto_Acha_Preco_pynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Bora Voar? Acha o Preço!**

A primeira instrução realiza as devidas instalações e importações ao projeto.

In [1]:
#Instala software development kit da Google /
!pip install -q -U google-generativeai #instala na shell via pip no modo silencioso (-q) e na versão mais recente (-U) a biblioteca:  google-generativeai

In [2]:
import google.generativeai as genai # Importa/ Carrega a biblioteca da IA Gemini da Google
import pandas as pd
import re
import textwrap   # Importa biblioteca de formato de texto
from IPython.display import display
from IPython.display import Markdown
from google.colab import userdata

Configuração da API Key do google AI Studio (Gemini) e dos parâmetros necessários.

In [3]:
GOOGLE_API_KEY=userdata.get("Secret_Key")
genai.configure(api_key=GOOGLE_API_KEY) #Configuração da chave de API informada

In [4]:
# Set up the model
generation_config = {
  "temperature": 0.9,
  "top_p": 1,
  "top_k": 0,
  "max_output_tokens": 2048,
}

In [5]:
safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
]


In [6]:
model = genai.GenerativeModel(model_name="gemini-1.0-pro",
                              generation_config=generation_config,
                              safety_settings=safety_settings)

In [7]:
chat = model.start_chat(history=[]) # guarda o histórico do chat

Pede para o usuário a inserção dos dados necessários e já estabelece o pront para ser enviado a AI.

In [10]:
# O modelo passará a coletar os dados necessários com o usuário
display(Markdown("# Bem vindo ao planejador de viagens mais acessíveis!"))

print("\nInforme os seguintes dados da sua viagem:") # Saúda o usuário
origem=input("\nOrigem: ") # Usuário informa a Origem do vôo
destino=input("Destino: ") #Usuário informa o destino do vôo
data_ida=input("Data da Viagem (pode ser uma data exata ou aproximada, mas não esqueça de infomar o ano!): ") # Data da viagem
retorno=int(input("Informe 1 caso seu vôo seja apenas de ida e 2 para ida e volta: ")) #Vôo apenas de Ida ou Ida e volta

while retorno!=1 and retorno!=2: # Caso haja digitação errada pelo usuário
        retorno=int(input("\n Desculpe, não entendi,poderia repetir? Informe 1 caso seu vôo seja apenas de ida e 2 para ida e volta: "))

if retorno==2: #Ou seja, prompt para o caso de passagem de ida e volta
        prompt_part = "Imagine que você é um passageiro e quer procurar passagens aéreas mais baratas, pesquise o site Google Flights e liste em formato de tabela os dados: data da viagem de ida - horário do vôo de ida - número do vôo de ida - Quantidade de conexões de ida - horário das conexões de ida -  quantidade de bagagens - companhia aerea da ida- data de retorno - horário de retorno- numero de voo de volta - conexões de volta- horario das conexões de volta - quantidade de bagagens de volta- companhia aérea da volta - preço total de uma viagem"
        data_retorno=input("Data do retorno (pode ser uma data exata ou aproximada): ") # Data da viagem
        novo_prompt = f"{prompt_part} de {origem} para {destino} em {data_ida} e retorno em {data_retorno}. O seu resultado deve estar disponível em uma tabela de 15 colunas "
else: #Ou seja, prompt para o caso de passagem de ida apenas
        prompt_part = "Imagine que você é um passageiro e quer procurar passagens aéreas mais baratas, pesquise o site Google Flights e liste em formato de tabela os dados: data da viagem - horário do vôo - número do vôo - Quantidade de conexões - horário das conexões -  quantidade de bagagens - companhia aérea- preço total de uma viagem em uma única tabela"
        novo_prompt = f"{prompt_part} de {origem} para {destino} em {data_ida}"

# Bem vindo ao planejador de viagens mais acessíveis!


Informe os seguintes dados da sua viagem:

Origem: Vitória-ES
Destino: Lisboa
Data da Viagem (pode ser uma data exata ou aproximada, mas não esqueça de infomar o ano!): agosto de 2024
Informe 1 caso seu vôo seja apenas de ida e 2 para ida e volta: 22

 Desculpe, não entendi,poderia repetir? Informe 1 caso seu vôo seja apenas de ida e 2 para ida e volta: 11

 Desculpe, não entendi,poderia repetir? Informe 1 caso seu vôo seja apenas de ida e 2 para ida e volta: 2
Data do retorno (pode ser uma data exata ou aproximada): dezembro de 2024


A IA sempre coloca marcadores separando o Cabeçalho e em eventuais testes, esses marcadores geraram erro por estarem em quantidade superior ao número de colunas, portanto o código os une para mais adiante serem removidos

In [13]:
novo_prompt2=str(novo_prompt)
response = chat.send_message(novo_prompt2)
texto=response.text
texto=re.sub(r'\|[-]+\|', '', texto)

Manipulação dos dados para se gerar um data_frame, como em cada teste realizado ocorreram pequenas variações no nome gerado pela IA para as colunas, elas passaram a ser fixadas com base na ordem solicitada para o retorno (prompt)

In [19]:
# Extrai as linhas da resposta
linhas = texto.strip().split("\n")

# Com base na nomenclatura do campo esperado, busca a linha em que começa o cabeçalho
indice1 = None
for i, linha in enumerate(linhas):
    if "| " in linha:
        indice1 = i
        break


# Extrai o cabeçalho
cabecalho = [header.strip() for header in linhas[i].split("|") if header.strip()]


# Extraindo os dados
dados = []
for linha in linhas[(i+1):]:
    colunas = [data.strip() for data in linha.split("|") if data.strip()]
    dados.append(colunas)

# Criando o DataFrame
df = pd.DataFrame(dados, columns=cabecalho)

#Exibe o título
texto_formatado = "<h1><b>Seguem os preços das menores passagens</b></h1>"
display(Markdown(texto_formatado))
print("\n")

if retorno==1: # Garante que o Cabeçalho sempre será o mesmo para viagens de ida
   df.columns = ["data_da_viagem","horario_do_voo", "número_vôo", "conexoes", "horario_conexoes", "quantidade_bagagens", "companhia_aerea","preco"]
elif retorno==2: # Garante que o Cabeçalho sempre será o mesmo para viagens de ida e volta
   df.columns = ["data_da_viagem","horario_do_voo_ida", "número_vôo_ida", "conexoes_ida", "horario_conexoes_ida", "bagagem_ida","companhia_aerea_ida", "data_retorno","horario_retorno","numero_voo_volta","conexoes_volta","horario_conexoes_volta","bagagem_volta","companhia_aerea","preco"]

# Exibindo o DataFrame como uma tabela
df = df.dropna(subset=["preco"]) #limpa os dados referentes a comentários e que por algum motivo estejam sem o preço
df = df[df["data_da_viagem"] != '---'] #limpa a divisória "---"

df #exibe o dataframe


<h1><b>Seguem os preços das menores passagens</b></h1>

,data_da_viagem,horario_do_voo_ida,número_vôo_ida,conexoes_ida,horario_conexoes_ida,bagagem_ida,companhia_aerea_ida,data_retorno,horario_retorno,numero_voo_volta,conexoes_volta,horario_conexoes_volta,bagagem_volta,companhia_aerea,preco
1,15/08/2024,13:55,LH4622,1,18:35 GRU,1,Lufthansa,15/12/2024,14:55,LH4704,1,19:15 FRA,1,Lufthansa,R$ 3.245
2,16/08/2024,17:55,TP106,0,-,2,TAP,16/12/2024,16:55,TP107,0,-,2,TAP,R$ 3.545
3,17/08/2024,11:55,AF4442,1,16:35 CDG,1,Air France,17/12/2024,13:55,AF4453,1,18:35 GRU,1,Air France,R$ 3.445
4,18/08/2024,18:55,BA539,0,-,2,British Airways,18/12/2024,15:55,BA540,0,-,2,British Airways,R$ 3.645
5,19/08/2024,14:55,KL745,1,19:35 AMS,1,KLM,19/12/2024,12:55,KL746,1,17:35 GRU,1,KLM,R$ 3.345
6,20/08/2024,10:55,AZ674,1,15:35 FCO,1,ITA Airways,20/12/2024,11:55,AZ675,1,16:35 GRU,1,ITA Airways,R$ 3.495
7,21/08/2024,16:55,UX1006,0,-,2,Air Europa,21/12/2024,14:55,UX1007,0,-,2,Air Europa,R$ 3.745
8,22/08/2024,12:55,IB3164,1,17:35 MAD,1,Iberia,22/12/2024,10:55,IB3165,1,15:35 GRU,1,Iberia,R$ 3.395
9,23/08/2024,18:55,LA2434,0,-,2,LATAM,23/12/2024,16:55,LA2435,0,-,2,LATAM,R$ 3.845
10,24/08/2024,14:55,JJ8784,1,19:35 GRU,1,Japan Airlines,24/12/2024,12:55,JJ8785,1,17:35 AMS,1,Japan Airlines,R$ 3.595


Detalha em texto o valor do menor preço e a companhia aérea que realiza o trajeto

In [15]:
df2 = df.copy()
df2["valor_monetario"] = df['preco'].str.replace(',', '.', regex=False).str.extract('(\d+\.?\d*)').astype(float)
index_min=df2["valor_monetario"].idxmin()

valor_minimo = df.loc[index_min,"preco"] #Encontra o menor valor da passagem:

display(Markdown("<b>Menor Valor da Passagem: </b>"))
print(valor_minimo)

if retorno==1:
  display(Markdown("<b>Disponível pela companhia: </b>"))
  Companhia_aerea = df.loc[index_min,"companhia_aerea"] # Retorna a companhia com o menor valor
  print(Companhia_aerea)
elif retorno==2:
  companhia_aerea_ida = df.loc[index_min,"companhia_aerea_ida"] # Retorna a companhia  de ida para o menor valor total
  companhia_aerea_retorno = df.loc[index_min,"companhia_aerea"] # Retorna a companhia de volta para o menor valor total
  display(Markdown("<b>Ida disponível pela companhia: </b>"))
  print(companhia_aerea_ida)
  display(Markdown("<b>Retorno disponível pela companhia: </b>"))
  print(companhia_aerea_retorno)

<b>Menor Valor da Passagem: </b>

R$ 3.245


<b>Ida disponível pela companhia: </b>

Lufthansa


<b>Retorno disponível pela companhia: </b>

Lufthansa


O Nosso Projeto passa então com base nos dados inseridos a sugerir eventos culturais para o usuário

In [18]:
#Exibe o título
texto_formatado2 = f"<h1><b>Enquanto estiver em {destino} seguem algumas sugestões: </b></h1>"
display(Markdown(texto_formatado2))
print("\n")
prompt = f"imagine que você é um entusiasta por atividades culturais, descreva em tópicos algumas atividades e locais históricos a serem visitados em {destino}, não esqueça de sugerir pratos típicos. Não separe por dias (pode ser que não coincida com os planos do cliente)"

response2 = chat.send_message(str(prompt))

display(Markdown(response2.text))




<h1><b>Enquanto estiver em Lisboa seguem algumas sugestões: </b></h1>

**Locais e Atividades Históricas em Lisboa:**

* **Castelo de São Jorge:** Explore as ruínas do castelo mourisco e desfrute de vistas panorâmicas da cidade a partir das muralhas medievais.
* **Sé de Lisboa (Catedral):** Visite a catedral mais antiga de Lisboa, datada do século XII, e admire sua arquitetura românica e gótica.
* **Mosteiro dos Jerónimos:** Maravilhe-se com a arquitetura manuelina deste Patrimônio Mundial da UNESCO e visite o túmulo de Vasco da Gama.
* **Torre de Belém:** Explore esta torre fortificada do século XVI, um símbolo da era dos descobrimentos portugueses.
* **Elevador de Santa Justa:** Suba neste elevador histórico para vistas deslumbrantes do centro da cidade de Lisboa.
* **Elétrico 28:** Faça um passeio no icônico elétrico 28, que percorre as ruas históricas e charmosas de Lisboa.
* **Museu Nacional dos Coches:** Admire uma coleção de carruagens reais e históricas, incluindo o famoso Coche de Ouro.

**Pratos Típicos para Experimentar:**

* **Bacalhau à Gomes de Sá:** Bacalhau salgado assado com batatas, cebolas, azeitonas e ovos.
* **Sardinhas Assadas:** Sardinhas grelhadas servidas com batatas cozidas e salada.
* **Pastéis de Bacalhau:** Bolinhos fritos recheados com bacalhau, cebola e salsa.
* **Arroz de Marisco:** Arroz cozido com uma variedade de frutos do mar, como camarão, mexilhões e lulas.
* **Cozido à Portuguesa:** Ensopado tradicional feito com vários tipos de carne, enchidos e vegetais.
* **Queijadas de Sintra:** Doces de queijo cremosos polvilhados com canela e açúcar.